### Import the necessary libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import requests
from io import BytesIO
from tqdm.notebook import tqdm
import pickle
import h5py
import nibabel as nib
from pycocotools.coco import COCO
from transformers import CLIPProcessor, CLIPModel

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the Model

In [2]:
# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Load the CLIP model for multimodal embeddings
print("Loading CLIP model for multimodal embeddings...")
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name)
clip_processor = CLIPProcessor.from_pretrained(model_name)

Loading CLIP model for multimodal embeddings...


In [ ]:
# ROI mapping dictionaries for better visualization and analysis
ROI_GROUPS = {
    'prf-visualrois': ['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4'],
    'floc-bodies': ['EBA', 'FBA-1', 'FBA-2', 'mTL-bodies'],
    'floc-faces': ['OFA', 'FFA-1', 'FFA-2', 'mTL-faces', 'aTL-faces'],
    'floc-places': ['OPA', 'PPA', 'RSC'],
    'floc-words': ['OWFA', 'VWFA-1', 'VWFA-2', 'mfs-words', 'mTL-words']
}

In [4]:
# Flatten ROI dictionary for easy access
ALL_ROIS = []
for rois in ROI_GROUPS.values():
    ALL_ROIS.extend(rois)

print(ALL_ROIS)

['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'EBA', 'FBA-1', 'FBA-2', 'mTL-bodies', 'OFA', 'FFA-1', 'FFA-2', 'mTL-faces', 'aTL-faces', 'OPA', 'PPA', 'RSC', 'OWFA', 'VWFA-1', 'VWFA-2', 'mfs-words', 'mTL-words']


In [ ]:
STIM_INFO_PATH = os.path.join("nsd_stim_info_merged.csv")

In [11]:
def load_stimulus_info():
    """
    Load and preprocess stimulus information from CSV
    """
    stim_info = pd.read_csv(STIM_INFO_PATH, index_col=0)
    important_columns = ['cocoId', 'cocoSplit', 'nsdId']
    stim_info = stim_info[important_columns]
    return stim_info



In [12]:
mapping_nsd_to_coco=load_stimulus_info()
mapping_nsd_to_coco.head()

,cocoId,cocoSplit,nsdId
0,532481,val2017,0
1,245764,val2017,1
2,385029,val2017,2
3,311303,val2017,3
4,393226,val2017,4


In [16]:
def get_coco_captions(nsdId, stim_info):
    """
    Get COCO captions for a given NSD image ID
    """
    try:
        # Get COCO ID and split
        coco_id = stim_info[stim_info['nsdId'] == nsdId]['cocoId'].values[0]
        coco_split = stim_info[stim_info['nsdId'] == nsdId]['cocoSplit'].values[0]
        
        # Load COCO annotation data
        coco_annotation_file = os.path.join(f"captions_{coco_split}.json")
        coco_data = COCO(coco_annotation_file)
        coco_ann_ids = coco_data.getAnnIds(coco_id)
        coco_annotations = coco_data.loadAnns(coco_ann_ids)
        
        # Extract captions
        captions = [anno['caption'] for anno in coco_annotations]
        return captions
    except Exception as e:
        print(f"Error getting captions for nsdId {nsdId}: {e}")
        return []

In [19]:
import re
def extract_nsd_id(image_path):
    """
    Extract nsd_id from the given image path.
    """
    match = re.search(r'nsd-(\d+)', image_path)
    if match:
        return int(match.group(1))
    else:
        raise ValueError("nsd_id not found in the image path")


In [21]:
ls subj04/roi_masks/

lh.all-vertices_fsaverage_space.npy    mapping_floc-words.npy
lh.floc-bodies_challenge_space.npy     mapping_prf-visualrois.npy
lh.floc-bodies_fsaverage_space.npy     mapping_streams.npy
lh.floc-faces_challenge_space.npy      rh.all-vertices_fsaverage_space.npy
lh.floc-faces_fsaverage_space.npy      rh.floc-bodies_challenge_space.npy
lh.floc-places_challenge_space.npy     rh.floc-bodies_fsaverage_space.npy
lh.floc-places_fsaverage_space.npy     rh.floc-faces_challenge_space.npy
lh.floc-words_challenge_space.npy      rh.floc-faces_fsaverage_space.npy
lh.floc-words_fsaverage_space.npy      rh.floc-places_challenge_space.npy
lh.prf-visualrois_challenge_space.npy  rh.floc-places_fsaverage_space.npy
lh.prf-visualrois_fsaverage_space.npy  rh.floc-words_challenge_space.npy
lh.streams_challenge_space.npy         rh.floc-words_fsaverage_space.npy
lh.streams_fsaverage_space.npy         rh.prf-visualrois_challenge_space.npy
mapping_floc-bodies.npy                rh.prf-visualrois_fsaverage_space.

In [ ]:
# Example usage
image_path = "subj04/training_split/training_images/train-0001_nsd-00012.png"
nsd_id = extract_nsd_id(image_path)
print(f"Extracted nsd_id: {nsd_id}")

# Assuming mapping_nsd_to_coco is defined elsewhere in your code
get_coco_captions(nsd_id, mapping_nsd_to_coco)

Extracted nsd_id: 12
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


['A man and woman bathe in a shallow river.',
 'A man and a woman in a river.',
 'A man brushing his teeth in a river while a woman looks on.',
 'A man and women in the river washing and eating.',
 'An Indian man and woman in the water on the edge of a river.']